https://www.youtube.com/watch?v=pGGjS6CkDeE&t=15s

In [17]:
import pandas as pd
from collections import namedtuple
from dataclasses import dataclass
import numpy as np

In [3]:
df = pd.DataFrame({
    'price': [10,20,30],
    'volume': [100,200,300],
}, index = pd.MultiIndex.from_arrays(["AAPL MSFT AAPL".split(), pd.date_range('2010-01-01', periods = 3)]))

In [4]:
print(df, sep='\n{}\n'.format('\N{box drawings light horizontal}' * 40))

                 price  volume
AAPL 2010-01-01     10     100
MSFT 2010-01-02     20     200
AAPL 2010-01-03     30     300


In [5]:
def market_value(df):
    ''''''

* def market_value(trades: list[dict[list...]]) etc is too strict and confusing
* his opinion that functions should only take in series and give out series
* programatically down the rows and structurally across the almost always when we do operations of pandas object

In [6]:
s = pd.Series(
    [10, 20, 30, 100*10, -200*20, 300*30],
    index = pd.MultiIndex.from_arrays(['AAPL MSFT AAPL USD USD USD'.split(), pd.date_range('2020-01-01', periods=6)], names = 'asset date'.split()))
print(s)
print(s.unstack('asset'))
print(s.unstack('asset').sum())

asset  date      
AAPL   2020-01-01      10
MSFT   2020-01-02      20
AAPL   2020-01-03      30
USD    2020-01-04    1000
       2020-01-05   -4000
       2020-01-06    9000
dtype: int64
asset       AAPL  MSFT     USD
date                          
2020-01-01  10.0   NaN     NaN
2020-01-02   NaN  20.0     NaN
2020-01-03  30.0   NaN     NaN
2020-01-04   NaN   NaN  1000.0
2020-01-05   NaN   NaN -4000.0
2020-01-06   NaN   NaN  9000.0
asset
AAPL      40.0
MSFT      20.0
USD     6000.0
dtype: float64


In [7]:
Trade = namedtuple('Trade', 'asset volume')

In [8]:
@dataclass
class Environment:
    prices: pd.Series
    def execute(self):
        pass


In [9]:
env = Environment(
    pd.Series([300, 400, 325, 425, 350, 450], index = pd.MultiIndex.from_product(['AAPL MSFT'.split(), pd.date_range('2010-01-01', periods=3)], names = 'asset date'.split()))
)

In [10]:
env

Environment(prices=asset  date      
AAPL   2010-01-01    300
       2010-01-02    400
       2010-01-03    325
MSFT   2010-01-01    425
       2010-01-02    350
       2010-01-03    450
dtype: int64)

In [30]:
prices = (
    pd.DataFrame(data ={
                'buy':[301, 401, 326, 426, 351, 451],
                'sell':[300, 400, 325, 425, 350, 450],
            },
            index = pd.MultiIndex.from_product(['AAPL MSFT'.split(), pd.date_range('2010-01-01', periods=3)], names = 'asset date'.split()))
).rename_axis(columns='direction')
prices

direction         buy  sell
asset date                 
AAPL  2010-01-01  301   300
      2010-01-02  401   400
      2010-01-03  326   325
MSFT  2010-01-01  426   425
      2010-01-02  351   350
      2010-01-03  451   450

In [13]:
trades = pd.Series([100,200,-150], 
                    index = pd.MultiIndex.from_arrays([['AAPL', 'MSFT', 'AAPL'], pd.date_range('2010-01-01', periods = 3)], 
                    names='asset date'.split()), 
                    name = 'volume')

In [14]:
trades

asset  date      
AAPL   2010-01-01    100
MSFT   2010-01-02    200
AAPL   2010-01-03   -150
Name: volume, dtype: int64

In [25]:
np.sign(trades).values

array([ 1,  1, -1], dtype=int64)

In [31]:
pd.MultiIndex.from_arrays([
    trades.index.get_level_values('asset'),
    trades.index.get_level_values('date'),
    pd.Series({+1:'buy',-1:'sell'}).loc[np.sign(trades).values]
], names = 'asset date direction'.split())

MultiIndex([('AAPL', '2010-01-01',  'buy'),
            ('MSFT', '2010-01-02',  'buy'),
            ('AAPL', '2010-01-03', 'sell')],
           names=['asset', 'date', 'direction'])

In [34]:
prices.stack('direction')

asset  date        direction
AAPL   2010-01-01  buy          301
                   sell         300
       2010-01-02  buy          401
                   sell         400
       2010-01-03  buy          326
                   sell         325
MSFT   2010-01-01  buy          426
                   sell         425
       2010-01-02  buy          351
                   sell         350
       2010-01-03  buy          451
                   sell         450
dtype: int64

In [40]:
prices.stack('direction').loc[
    pd.MultiIndex.from_arrays([
    trades.index.get_level_values('asset'),
    trades.index.get_level_values('date'),
    pd.Series({+1:'buy',-1:'sell'}).loc[np.sign(trades).values]
], names = 'asset date direction'.split())
] * trades

asset  date        direction  portfolio
AAPL   2010-01-01  buy        Alice        30100
MSFT   2010-01-02  buy        Bob          70200
AAPL   2010-01-03  sell       Alice       -48750
dtype: int64

what if the source index had extra index layers?

In [39]:
trades = pd.Series([100,200,-150], 
                    index = pd.MultiIndex.from_arrays(['Alice Bob Alice'.split(), ['AAPL', 'MSFT', 'AAPL'], pd.date_range('2010-01-01', periods = 3)], 
                    names='portfolio asset date'.split()), 
                    name = 'volume')
trades

portfolio  asset  date      
Alice      AAPL   2010-01-01    100
Bob        MSFT   2010-01-02    200
Alice      AAPL   2010-01-03   -150
Name: volume, dtype: int64

What causes this to be slow?

* Look up with multiindex very slow (very easy to code up in C however)
* The structure is already index aligned from the .loc so we can just work with the values?

In [45]:
pd.concat([ trades, trades * prices.stack('direction').loc[
    pd.MultiIndex.from_arrays([
        trades.index.get_level_values('asset'),
        trades.index.get_level_values('date'),
        pd.Series({+1:'buy',-1:'sell'}).loc[np.sign(trades).values]
    ], names = 'asset date direction'.split())
].values]).set_axis(

)

portfolio  asset  date      
Alice      AAPL   2010-01-01      100
Bob        MSFT   2010-01-02      200
Alice      AAPL   2010-01-03     -150
                  2010-01-01    30100
Bob        MSFT   2010-01-02    70200
Alice      AAPL   2010-01-03   -48750
Name: volume, dtype: int64